In [1]:
import uuid
import httpx
import hashlib
import hmac

import time
from datetime import datetime
import json

import pandas as pd
import numpy as np

In [2]:
default_quantity: float = 1.00,
api_url: str = "https://api-demo.bybit.com"
api_key: str = "6TzoOeOuyIDiJN9cOu"
api_secret: str = "JbTE9Z5OFFyxcw1mzqf6hLIpmGgt8dONmlSE"
# Static Values for API URL

endpoint_Place_Order = "/v5/order/create"

endpoint_get_position_info = "/v5/position/list"

# Static Values for API Key
API_Key = api_key

# Static Values for API Secret
API_Secret = api_secret

# Minimum Order Quantity for Perpetual Futures
min_qty_perpFutures = 0.001

# Minimum Order Quantity for Perpetual Futures
min_qty_Options = 0.01

# Convert the Options Base Shorting Quantity into String as it needs to be passed in Params as a string object;
# it's expressed in Lot Size, Bitcoin lot size is 1.00 BTC for 1000 Contracts so its 0.001 Per Contract
default_quantity: str = str(default_quantity)

# Static Parameters
recv_window: str = str(5000)

category: dict = {"spot": "spot",
                       "linear": "linear",
                       "inverse": "inverse",
                       "option": "option"}

side: dict = {"Buy": "Buy", "Sell": "Sell"}

orderType: dict = {"Market": "Market", "Limit": "Limit"}

timeInForce: dict = {"IOC": "IOC", "GTC": "GTC"}

BTC_perpetual_future: str = "BTCPERP"

baseCoin = "BTC"
settleCoin = "USDC"

optionParams: dict = {}
json_optionParams: str | None = None
PerpFutureParams: dict = {}
jsonPerpFutureParams: str | None = None

options_position_params: dict = {}
json_options_position_params: str | None = None

perp_future_position_params: dict = {}
json_perp_future_position_params: str | None = None

option_position_params: dict = {}
json_option_position_params: str | None = None

####
option_position_info_json = None
options_position_dataframe: pd.DataFrame | None = None

In [3]:
async def _generateSignature(params, time_stamp):
    """
    :param params: Its API Parameter in Dict → JSON → String Format
    :param time_stamp: Its System generated Time module
    :return: ByBit API Signature
    """
    param_str = str(time_stamp) + API_Key + recv_window + str(params)
    print(f"Parameter String is : {param_str}\n")
    ByBit_hash = hmac.new(bytes(API_Secret, "utf-8"), param_str.encode("utf-8"), hashlib.sha256)
    signature = ByBit_hash.hexdigest()
    print(f"Generated Signature is : {signature}\n")
    return signature

In [4]:
async def get_option_positions(category: str = "option",
                               baseCoin: str = "BTC",
                               settleCoin: str = "USDC"):

    btc_option_position_params = f"category={category}" + f"&baseCoin={baseCoin}" + f"&settleCoin={settleCoin}"

    time_stamp: str = str(int(time.time() * 10 ** 3))

    option_api_signature = await _generateSignature(params=btc_option_position_params,
                                                         time_stamp=time_stamp)

    print(f"BTC Option Position Info API Signature : {option_api_signature}\n")

    btc_option_Headers: dict = {
        'X-BAPI-API-KEY': API_Key,
        'X-BAPI-SIGN': option_api_signature,
        'X-BAPI-SIGN-TYPE': '2',
        'X-BAPI-TIMESTAMP': time_stamp,
        'X-BAPI-RECV-WINDOW': recv_window,
        'Content-Type': 'application/json'
    }

    print(f"API Header is : {btc_option_Headers} \n")

    url = f"{api_url}" + f"{endpoint_get_position_info}" + f"?" + f"{btc_option_position_params}"

    print(f"Final URL is : {url}")

    try:
        async with httpx.AsyncClient() as client:
            # Await Data from the "GET" Request
            option_position_info_json = await client.get(url=url, headers=btc_option_Headers)
            # Check if We have Received the Data or its Data Error
        print(f"Positions Info Length is : {len(option_position_info_json.json()["result"]["list"])}")
    except Exception as e:
        # Print the Error Message if No Data is Received
        print(f"Error in Fetching the Positions Info, Error Details : Exception : {e}")

    return option_position_info_json

In [5]:
position_json = await get_option_positions()

Parameter String is : 17298821052266TzoOeOuyIDiJN9cOu5000category=option&baseCoin=BTC&settleCoin=USDC

Generated Signature is : 410d446066f8e182b983e326b5444cacf1758fc12e1f9d9cbf6756eebdc3fb4b

BTC Option Position Info API Signature : 410d446066f8e182b983e326b5444cacf1758fc12e1f9d9cbf6756eebdc3fb4b

API Header is : {'X-BAPI-API-KEY': '6TzoOeOuyIDiJN9cOu', 'X-BAPI-SIGN': '410d446066f8e182b983e326b5444cacf1758fc12e1f9d9cbf6756eebdc3fb4b', 'X-BAPI-SIGN-TYPE': '2', 'X-BAPI-TIMESTAMP': '1729882105226', 'X-BAPI-RECV-WINDOW': '5000', 'Content-Type': 'application/json'} 

Final URL is : https://api-demo.bybit.com/v5/position/list?category=option&baseCoin=BTC&settleCoin=USDC
Positions Info Length is : 16


In [6]:
options_position_dataframe = pd.json_normalize(data=position_json.json()["result"]["list"])

In [7]:
positions_labels = ["leverage", "autoAddMargin", "riskLimitValue", "takeProfit", "isReduceOnly",
                    "tpslMode", "leverageSysUpdatedTime", "mmrSysUpdatedTime", "stopLoss", "tradeMode",
                    "sessionAvgPrice", "trailingStop", "bustPrice", "positionBalance", "positionIdx",
                    "positionIM", "positionMM", "adlRankIndicator", "cumRealisedPnl", "riskId",
                    "liqPrice"]

ByBit_date_format = "%d%b%y"  # Date Format as (DD)(MMM)(YY)

options_position_dataframe.drop(labels=positions_labels, axis=1, inplace=True)

In [8]:
# We are Initialing Filling "UNKNOWN" as String Value, Later it will be replaced by "CALL" or "PUT"
option_type = np.array(["UNKNOWN" for _ in range(len(options_position_dataframe))])

# Initialize Strike_Price Numpy Array, we need to fill it with actual Values after Splitting the
# Symbol String We are Initialing Filling "-5000" as Float64 Value, Later it will be replaced by
# Actual Strike Price
strike_price = np.array([-5000.00 for _ in range(len(options_position_dataframe))])

# Initialize Expiry_Date Numpy Array; we need to fill it with actual Values after Splitting the
# Symbol String We are Initialing Filling "1-1-2000" as datetime Value, Later it will be replaced by
# Actual Expiry Dates
expiry = np.array([datetime(year=2000, month=1, day=1).date() for _ in range(len(options_position_dataframe))])

# Initialize Positional_Delta Numpy Array,
# we need to fill it with actual Values after Computing the Delta of PUT & CALL Options
# We are Initialing Filling "-5000" as Float64 Value, Later it will be replaced by Computed Delta Value
total_delta = np.array([-5000.00 for _ in range(len(options_position_dataframe))])

# We are Initialing Filling "-5000" as Float64 Value, Later it will be replaced by Computed Delta Value
expiry_delta = np.array([-5000.00 for _ in range(len(options_position_dataframe))])

# Define the NP Array for the Type of Expiry "Daily", "Weekly", "Monthly", "Quarterly"
expiry_type = np.array(["UNKNOWN" for _ in range(len(options_position_dataframe))])

# Define the Tolerance Limit of Delta before Executing the Hedging
delta_hedging = np.array([-5000.00 for _ in range(len(options_position_dataframe))])

# Define the Bool Array to Signify the Delta Hedging Requirements
hedging_required = np.array([False for _ in range(len(options_position_dataframe))])

# Define the NP Array to Compute Perpectual Futures Required to fully Delta Hedge this Position
PerpFutureQty = np.array([-5000.00 for _ in range(len(options_position_dataframe))])

################

# Merge the Numpy Array of "Option Type" with Primary DataFrame
options_position_dataframe['option_type'] = option_type

# Merge the Numpy Array of "Strike_Price" with Primary DataFrame
options_position_dataframe['strike_price'] = strike_price

# Merge the Numpy Array of "Expiry Dates" with Primary DataFrame
options_position_dataframe['expiry'] = expiry

# Merge the Numpy Array of "Total Delta" with Primary DataFrame
options_position_dataframe['total_delta'] = total_delta

# Merge the Numpy Array of "Expiry Delta" with Primary DataFrame
options_position_dataframe['expiry_delta'] = expiry_delta

# Define the NP Array for the Type of Expiry "Daily", "Weekly", "Monthly", "Quarterly"
options_position_dataframe['expiry_type'] = expiry_type

# Define the Tolerance Limit of Delta before Executing the Hedging
options_position_dataframe['delta_hedging'] = delta_hedging

# Define the Bool Array to Signify the Delta Hedging Requirements
options_position_dataframe['hedging_required'] = hedging_required

# Define the NP Array to Compute Perpectual Futures Required to fully Delta Hedge this Position
options_position_dataframe['PerpFutureQty'] = PerpFutureQty

In [9]:
try:
    options_position_dataframe['symbol'] = options_position_dataframe['symbol'].astype(dtype="string")
except Exception as e:
    print(f"Error in processing 'symbol' dataType, Error Code : {e}")

try:
    options_position_dataframe['avgPrice'] = options_position_dataframe['avgPrice'].astype(dtype="float64")
except Exception as e:
    print(f"Error in processing 'avgPrice' dataType, Error Code : {e}")

try:
    options_position_dataframe['delta'] = options_position_dataframe['delta'].astype(dtype="float64")
except Exception as e:
    print(f"Error in processing 'delta' dataType, Error Code : {e}")

try:
    options_position_dataframe['theta'] = options_position_dataframe['theta'].astype(dtype="float64")
except Exception as e:
    print(f"Error in processing 'theta' dataType, Error Code : {e}")

try:
    options_position_dataframe['positionValue'] = options_position_dataframe['positionValue'].astype(dtype="float64")
except Exception as e:
    print(f"Error in processing 'positionValue' dataType, Error Code : {e}")

try:
    options_position_dataframe['unrealisedPnl'] = options_position_dataframe['unrealisedPnl'].astype(dtype="float64")
except Exception as e:
    print(f"Error in processing 'unrealisedPnl' dataType, Error Code : {e}")

try:
    options_position_dataframe['markPrice'] = options_position_dataframe['markPrice'].astype(dtype="float64")
except Exception as e:
    print(f"Error in processing 'markPrice' dataType, Error Code : {e}")

try:
    options_position_dataframe['createdTime'] = options_position_dataframe['createdTime'].astype(dtype="float64")
    options_position_dataframe['createdTime'] = pd.to_datetime(options_position_dataframe['createdTime'], unit='ms')
except Exception as e:
    print(f"Error in processing 'createdTime' dataType, Error Code : {e}")

try:
    options_position_dataframe['seq'] = options_position_dataframe['seq'].astype(dtype="int64")
except Exception as e:
    print(f"Error in processing 'seq' dataType, Error Code : {e}")

try:
    options_position_dataframe['updatedTime'] = options_position_dataframe['updatedTime'].astype(dtype="float64")
    options_position_dataframe['updatedTime'] = pd.to_datetime(options_position_dataframe['updatedTime'], unit='ms')
except Exception as e:
    print(f"Error in processing 'updatedTime' dataType, Error Code : {e}")

try:
    options_position_dataframe['side'] = options_position_dataframe['side'].astype(dtype="string")
except Exception as e:
    print(f"Error in processing 'side' dataType, Error Code : {e}")

try:
    options_position_dataframe['curRealisedPnl'] = options_position_dataframe['curRealisedPnl'].astype(dtype="float64")
except Exception as e:
    print(f"Error in processing 'curRealisedPnl' dataType, Error Code : {e}")

try:
    options_position_dataframe['size'] = options_position_dataframe['size'].astype(dtype="float64")
except Exception as e:
    print(f"Error in processing 'size' dataType, Error Code : {e}")

try:
    options_position_dataframe['positionStatus'] = options_position_dataframe['positionStatus'].astype(dtype="string")
except Exception as e:
    print(f"Error in processing 'positionStatus' dataType, Error Code : {e}")

try:
    options_position_dataframe['gamma'] = options_position_dataframe['gamma'].astype(dtype="float64")
except Exception as e:
    print(f"Error in processing 'gamma' dataType, Error Code : {e}")

try:
    options_position_dataframe['vega'] = options_position_dataframe['vega'].astype(dtype="float64")
except Exception as e:
    print(f"Error in processing 'vega' dataType, Error Code : {e}")

try:
    options_position_dataframe['option_type'] = options_position_dataframe['option_type'].astype(dtype="string")
except Exception as e:
    print(f"Error in processing 'option_type' dataType, Error Code : {e}")

try:
    options_position_dataframe['strike_price'] = options_position_dataframe['strike_price'].astype(dtype="float64")
except Exception as e:
    print(f"Error in processing 'strike_price' dataType, Error Code : {e}")

try:
    options_position_dataframe['total_delta'] = options_position_dataframe['total_delta'].astype(dtype="float64")
except Exception as e:
    print(f"Error in processing 'total_delta' dataType, Error Code : {e}")

try:
    options_position_dataframe['expiry_delta'] = options_position_dataframe['expiry_delta'].astype(dtype="float64")
except Exception as e:
    print(f"Error in processing 'expiry_delta' dataType, Error Code : {e}")

try:
    options_position_dataframe['expiry_type'] = options_position_dataframe['expiry_type'].astype(dtype="string")
except Exception as e:
    print(f"Error in processing 'expiry_type' dataType, Error Code : {e}")

try:
    options_position_dataframe['delta_hedging'] = options_position_dataframe['delta_hedging'].astype(dtype="float64")
except Exception as e:
    print(f"Error in processing 'delta_hedging' dataType, Error Code : {e}")

try:
    options_position_dataframe['hedging_required'] = options_position_dataframe['hedging_required'].astype(dtype="bool")
except Exception as e:
    print(f"Error in processing 'hedging_required' dataType, Error Code : {e}")

try:
    options_position_dataframe['PerpFutureQty'] = options_position_dataframe['PerpFutureQty'].astype(dtype="float64")
except Exception as e:
    print(f"Error in processing 'PerpFutureQty' dataType, Error Code : {e}")


In [10]:
options_position_dataframe['updatedTime'], options_position_dataframe['createdTime']

(0    2024-10-25 12:35:02.856
 1    2024-10-25 12:35:02.480
 2    2024-10-24 17:45:02.810
 3    2024-10-24 17:45:02.610
 4    2024-10-24 17:45:02.302
 5    2024-10-24 17:45:01.938
 6    2024-10-18 15:47:31.097
 7    2024-10-18 15:47:30.703
 8    2024-10-18 06:00:38.554
 9    2024-10-18 06:00:38.347
 10   2024-10-18 06:00:38.007
 11   2024-10-18 06:00:37.732
 12   2024-10-18 06:00:37.378
 13   2024-10-18 06:00:37.172
 14   2024-09-20 05:19:08.362
 15   2024-09-20 05:19:08.122
 Name: updatedTime, dtype: datetime64[ns],
 0    2024-10-25 12:35:02.855
 1    2024-10-25 12:35:02.479
 2    2024-10-24 17:45:02.810
 3    2024-10-24 17:45:02.610
 4    2024-10-24 17:45:02.301
 5    2024-10-24 17:45:01.938
 6    2024-10-18 15:47:31.096
 7    2024-10-18 15:47:30.703
 8    2024-10-18 06:00:38.554
 9    2024-10-18 06:00:38.346
 10   2024-10-18 06:00:38.007
 11   2024-10-18 06:00:37.732
 12   2024-10-18 06:00:37.377
 13   2024-10-18 06:00:37.171
 14   2024-09-20 05:19:07.715
 15   2024-09-20 05:19:07.3

In [11]:
for i in range(len(options_position_dataframe)):
    split_symbol = options_position_dataframe.loc[i, "symbol"].split("-")

    options_position_dataframe.loc[i, "expiry"] = datetime.strptime(str(split_symbol[1]), ByBit_date_format).date()

    options_position_dataframe.loc[i, "strike_price"] = float(split_symbol[2])

    if split_symbol[3] == "P":
        options_position_dataframe.loc[i, "option_type"] = "PUT"

    elif split_symbol[3] == "C":
        options_position_dataframe.loc[i, "option_type"] = "CALL"
    
    options_position_dataframe.loc[i, "total_delta"] = (float(options_position_dataframe.iloc[i]["size"]) * 
                                                             float(options_position_dataframe.iloc[i]["delta"]))
    
    print(f"For Index Position {i}, Symbol Data was Split Correctly : {split_symbol}",
          f"Expiry Data: {options_position_dataframe.loc[i, "expiry"]}",
          f"Strike Price: {options_position_dataframe.loc[i, "strike_price"]}",
          f"Options Type Data: {options_position_dataframe.loc[i, "option_type"]}",
          f"Total Delta : {options_position_dataframe.loc[i, "total_delta"]} , : {options_position_dataframe.loc[i,"delta"]}\n")
    

For Index Position 0, Symbol Data was Split Correctly : ['BTC', '28OCT24', '65000', 'P'] Expiry Data: 2024-10-28 Strike Price: 65000.0 Options Type Data: PUT Total Delta : 0.11603081 , : 0.11603081

For Index Position 1, Symbol Data was Split Correctly : ['BTC', '28OCT24', '72000', 'C'] Expiry Data: 2024-10-28 Strike Price: 72000.0 Options Type Data: CALL Total Delta : -0.02429658 , : -0.02429658

For Index Position 2, Symbol Data was Split Correctly : ['BTC', '27OCT24', '64000', 'P'] Expiry Data: 2024-10-27 Strike Price: 64000.0 Options Type Data: PUT Total Delta : 0.0373424 , : 0.0373424

For Index Position 3, Symbol Data was Split Correctly : ['BTC', '27OCT24', '72000', 'C'] Expiry Data: 2024-10-27 Strike Price: 72000.0 Options Type Data: CALL Total Delta : -0.00741585 , : -0.00741585

For Index Position 4, Symbol Data was Split Correctly : ['BTC', '26OCT24', '65000', 'P'] Expiry Data: 2024-10-26 Strike Price: 65000.0 Options Type Data: PUT Total Delta : 0.03395151 , : 0.03395151

F

In [12]:
options_position_dataframe["delta"], options_position_dataframe["size"], options_position_dataframe["total_delta"]

(0     0.116031
 1    -0.024297
 2     0.037342
 3    -0.007416
 4     0.033952
 5    -0.005200
 6     0.012409
 7    -0.021254
 8     0.057470
 9    -0.050414
 10    0.039483
 11   -0.034816
 12    0.017968
 13   -0.008597
 14    0.011633
 15   -0.016221
 Name: delta, dtype: float64,
 0     1.0
 1     1.0
 2     1.0
 3     1.0
 4     1.0
 5     1.0
 6     1.0
 7     1.0
 8     1.0
 9     1.0
 10    1.0
 11    1.0
 12    1.0
 13    1.0
 14    2.0
 15    2.0
 Name: size, dtype: float64,
 0     0.116031
 1    -0.024297
 2     0.037342
 3    -0.007416
 4     0.033952
 5    -0.005200
 6     0.012409
 7    -0.021254
 8     0.057470
 9    -0.050414
 10    0.039483
 11   -0.034816
 12    0.017968
 13   -0.008597
 14    0.023266
 15   -0.032441
 Name: total_delta, dtype: float64)

In [13]:
sum(options_position_dataframe["total_delta"])

0.15348621999999998

In [14]:
pd.to_datetime(options_position_dataframe['createdTime'], unit='ms')

0    2024-10-25 12:35:02.855
1    2024-10-25 12:35:02.479
2    2024-10-24 17:45:02.810
3    2024-10-24 17:45:02.610
4    2024-10-24 17:45:02.301
5    2024-10-24 17:45:01.938
6    2024-10-18 15:47:31.096
7    2024-10-18 15:47:30.703
8    2024-10-18 06:00:38.554
9    2024-10-18 06:00:38.346
10   2024-10-18 06:00:38.007
11   2024-10-18 06:00:37.732
12   2024-10-18 06:00:37.377
13   2024-10-18 06:00:37.171
14   2024-09-20 05:19:07.715
15   2024-09-20 05:19:07.372
Name: createdTime, dtype: datetime64[ns]

In [15]:
async def _get_time_stamp():
    time_stamp: str = str(int(time.time() * 10 ** 3))
    return time_stamp

In [16]:

async def get_PerpFutures_Position():
    perpfutures_params = (f"category={category["linear"]}" +
                          f"&baseCoin={baseCoin}" +
                          f"&settleCoin={settleCoin}")

    time_stamp = await _get_time_stamp()

    perpfutures_api_sigs = await _generateSignature(params=perpfutures_params,
                                                         time_stamp=time_stamp)

    print(f"Perpectual Futures Position Info API Signature : {perpfutures_api_sigs}\n")

    perpfutures_Headers: dict = {
        'X-BAPI-API-KEY': API_Key,
        'X-BAPI-SIGN': perpfutures_api_sigs,
        'X-BAPI-SIGN-TYPE': '2',
        'X-BAPI-TIMESTAMP': time_stamp,
        'X-BAPI-RECV-WINDOW': recv_window,
        'Content-Type': 'application/json'
    }

    print(f"API Header is : {perpfutures_Headers} \n")

    url = f"{api_url}" + f"{endpoint_get_position_info}" + f"?" + f"{perpfutures_params}"

    print(f"Final URL is : {url}")

    try:
        async with httpx.AsyncClient() as client:
            # Await Data from the "GET" Request
            perpfutures_position_info_json = await client.get(url=url, headers=perpfutures_Headers)
            # Check if We have Received the Data or its Data Error
        print(f"Positions Info Length is : {len(perpfutures_position_info_json.json()["result"]["list"])}")
    except Exception as e:
        # Print the Error Message if No Data is Received
        print(f"Error in Fetching the Positions Info, Error Details : {e}")

    return option_position_info_json

In [17]:
await get_PerpFutures_Position()

Parameter String is : 17298821208226TzoOeOuyIDiJN9cOu5000category=linear&baseCoin=BTC&settleCoin=USDC

Generated Signature is : 0a2c4cb04b4c8bf3df8727e291fb6ca69a9efab2b15ddb50f5f9aae34693b44c

Perpectual Futures Position Info API Signature : 0a2c4cb04b4c8bf3df8727e291fb6ca69a9efab2b15ddb50f5f9aae34693b44c

API Header is : {'X-BAPI-API-KEY': '6TzoOeOuyIDiJN9cOu', 'X-BAPI-SIGN': '0a2c4cb04b4c8bf3df8727e291fb6ca69a9efab2b15ddb50f5f9aae34693b44c', 'X-BAPI-SIGN-TYPE': '2', 'X-BAPI-TIMESTAMP': '1729882120822', 'X-BAPI-RECV-WINDOW': '5000', 'Content-Type': 'application/json'} 

Final URL is : https://api-demo.bybit.com/v5/position/list?category=linear&baseCoin=BTC&settleCoin=USDC
Positions Info Length is : 1


In [18]:
options_position_dataframe

,symbol,avgPrice,delta,theta,positionValue,unrealisedPnl,markPrice,createdTime,seq,updatedTime,...,vega,option_type,strike_price,expiry,total_delta,expiry_delta,expiry_type,delta_hedging,hedging_required,PerpFutureQty
0,BTC-28OCT24-65000-P,30.0,0.116031,77.243886,-113.320494,-83.320494,113.320494,2024-10-25 12:35:02.855,671313656,2024-10-25 12:35:02.856,...,-10.993850,PUT,65000.0,2024-10-28,0.116031,-5000.0,UNKNOWN,-5000.0,False,-5000.0
1,BTC-28OCT24-72000-C,25.0,-0.024297,25.358818,-21.009976,3.990024,21.009976,2024-10-25 12:35:02.479,671313655,2024-10-25 12:35:02.480,...,-3.211538,CALL,72000.0,2024-10-28,-0.024297,-5000.0,UNKNOWN,-5000.0,False,-5000.0
2,BTC-27OCT24-64000-P,35.0,0.037342,50.620086,-28.271145,6.728855,28.271145,2024-10-24 17:45:02.810,671306217,2024-10-24 17:45:02.810,...,-3.574202,PUT,64000.0,2024-10-27,0.037342,-5000.0,UNKNOWN,-5000.0,False,-5000.0
3,BTC-27OCT24-72000-C,35.0,-0.007416,12.215779,-4.615143,30.384857,4.615143,2024-10-24 17:45:02.610,671306216,2024-10-24 17:45:02.610,...,-0.899454,CALL,72000.0,2024-10-27,-0.007416,-5000.0,UNKNOWN,-5000.0,False,-5000.0
4,BTC-26OCT24-65000-P,50.0,0.033952,48.596561,-16.810165,33.189835,16.810165,2024-10-24 17:45:02.301,671306215,2024-10-24 17:45:02.302,...,-1.968472,PUT,65000.0,2024-10-26,0.033952,-5000.0,UNKNOWN,-5000.0,False,-5000.0
5,BTC-26OCT24-71000-C,40.0,-0.005200,10.421973,-2.265604,37.734396,2.265604,2024-10-24 17:45:01.938,671306214,2024-10-24 17:45:01.938,...,-0.391220,CALL,71000.0,2024-10-26,-0.005200,-5000.0,UNKNOWN,-5000.0,False,-5000.0
6,BTC-28MAR25-30000-P,215.0,0.012409,3.059205,-144.597463,70.402537,144.597463,2024-10-18 15:47:31.096,671276734,2024-10-18 15:47:31.097,...,-14.617901,PUT,30000.0,2025-03-28,0.012409,-5000.0,UNKNOWN,-5000.0,False,-5000.0
7,BTC-28MAR25-200000-C,250.0,-0.021254,5.396963,-221.159279,28.840721,221.159279,2024-10-18 15:47:30.703,671276733,2024-10-18 15:47:30.703,...,-23.176938,CALL,200000.0,2025-03-28,-0.021254,-5000.0,UNKNOWN,-5000.0,False,-5000.0
8,BTC-29NOV24-52000-P,455.0,0.057470,20.389698,-321.471386,133.528614,321.471386,2024-10-18 06:00:38.554,671276193,2024-10-18 06:00:38.554,...,-24.077111,PUT,52000.0,2024-11-29,0.057470,-5000.0,UNKNOWN,-5000.0,False,-5000.0
9,BTC-29NOV24-98000-C,305.0,-0.050414,21.412478,-279.820803,25.179197,279.820803,2024-10-18 06:00:38.346,671276192,2024-10-18 06:00:38.347,...,-21.704996,CALL,98000.0,2024-11-29,-0.050414,-5000.0,UNKNOWN,-5000.0,False,-5000.0


In [ ]:
positions_expiry_list = options_position_dataframe["expiry"].drop_duplicates().copy()

# numpy_positions_expiry_list = self.options_position_dataframe["expiry"].drop_duplicates().to_numpy()
try:
    for i in range(len(positions_expiry_list)):
        index_value = int(positions_expiry_list.index[i])
        print(f"The Index Value Type is : {type(index_value)} , & the Index Value is : {index_value}")
        cumulative_delta = sum(options_position_dataframe[options_position_dataframe.expiry ==
                                                               positions_expiry_list[index_value]].total_delta)
        print(f"Cumulative_Delta is : {cumulative_delta}")
        options_position_dataframe.loc[
            options_position_dataframe.expiry == positions_expiry_list[
                index_value], "delta_hedging"] = cumulative_delta


except Exception as e:
    print(f"Error in processing 'Cumulative Delta', Error Code : {e}")

In [ ]:
positions_expiry_list

In [ ]:
options_position_dataframe

In [ ]:
0.087150 + ( - 0.050502)

In [19]:
from ByBit import ByBitAPI

In [20]:
ByBitAPI = ByBitAPI(default_quantity = 1.00,
                    api_url = "https://api-demo.bybit.com",
                    api_key = "6TzoOeOuyIDiJN9cOu",
                    api_secret = "JbTE9Z5OFFyxcw1mzqf6hLIpmGgt8dONmlSE",
                    baseCoin = "BTC",
                   settleCoin = "USDC")

In [21]:
json_file = await ByBitAPI.get_option_positions()

Parameter String is : 17298821506716TzoOeOuyIDiJN9cOu5000category=option&baseCoin=BTC&settleCoin=USDC

Generated Signature is : 083c5b303167a0176a6018add52469b81918ca4d976449c51d9f47a8e62e0a71

Option Position Info API Signature : 083c5b303167a0176a6018add52469b81918ca4d976449c51d9f47a8e62e0a71

API Header is : {'X-BAPI-API-KEY': '6TzoOeOuyIDiJN9cOu', 'X-BAPI-SIGN': '083c5b303167a0176a6018add52469b81918ca4d976449c51d9f47a8e62e0a71', 'X-BAPI-SIGN-TYPE': '2', 'X-BAPI-TIMESTAMP': '1729882150671', 'X-BAPI-RECV-WINDOW': '5000', 'Content-Type': 'application/json'} 

Final URL is : https://api-demo.bybit.com/v5/position/list?category=option&baseCoin=BTC&settleCoin=USDC
Positions Info Length is : 16


In [22]:
dataframe_file = await ByBitAPI.format_option_position_dataframe()

For Index Position 0, Symbol Data was Split Correctly : ['BTC', '28OCT24', '65000', 'P'] Expiry Data: 2024-10-28 Strike Price: 65000.0 Options Type Data: PUT,Total Delta : 0.12109579 ,Delta : 0.12109579 

For Index Position 1, Symbol Data was Split Correctly : ['BTC', '28OCT24', '72000', 'C'] Expiry Data: 2024-10-28 Strike Price: 72000.0 Options Type Data: CALL,Total Delta : -0.02253647 ,Delta : -0.02253647 

For Index Position 2, Symbol Data was Split Correctly : ['BTC', '27OCT24', '64000', 'P'] Expiry Data: 2024-10-27 Strike Price: 64000.0 Options Type Data: PUT,Total Delta : 0.0383938 ,Delta : 0.03839380 

For Index Position 3, Symbol Data was Split Correctly : ['BTC', '27OCT24', '72000', 'C'] Expiry Data: 2024-10-27 Strike Price: 72000.0 Options Type Data: CALL,Total Delta : -0.00753321 ,Delta : -0.00753321 

For Index Position 4, Symbol Data was Split Correctly : ['BTC', '26OCT24', '65000', 'P'] Expiry Data: 2024-10-26 Strike Price: 65000.0 Options Type Data: PUT,Total Delta : 0.0

In [23]:
dataframe_file

,symbol,avgPrice,delta,theta,positionValue,unrealisedPnl,markPrice,createdTime,seq,updatedTime,...,vega,option_type,strike_price,expiry,total_delta,expiry_delta,expiry_type,delta_hedging,hedging_required,PerpFutureQty
0,BTC-28OCT24-65000-P,30.0,0.121096,79.497965,-123.189282,-93.189282,123.189282,2024-10-25 12:35:02.855,671313656,2024-10-25 12:35:02.856,...,-11.320828,PUT,65000.0,2024-10-28,0.121096,-5000.0,UNKNOWN,0.098559,False,-5000.0
1,BTC-28OCT24-72000-C,25.0,-0.022536,23.692884,-18.467410,6.532590,18.467410,2024-10-25 12:35:02.479,671313655,2024-10-25 12:35:02.480,...,-3.011945,CALL,72000.0,2024-10-28,-0.022536,-5000.0,UNKNOWN,0.098559,False,-5000.0
2,BTC-27OCT24-64000-P,35.0,0.038394,51.300760,-30.116805,4.883195,30.116805,2024-10-24 17:45:02.810,671306217,2024-10-24 17:45:02.810,...,-3.652624,PUT,64000.0,2024-10-27,0.038394,-5000.0,UNKNOWN,0.030861,False,-5000.0
3,BTC-27OCT24-72000-C,35.0,-0.007533,12.555579,-4.513926,30.486074,4.513926,2024-10-24 17:45:02.610,671306216,2024-10-24 17:45:02.610,...,-0.911112,CALL,72000.0,2024-10-27,-0.007533,-5000.0,UNKNOWN,0.030861,False,-5000.0
4,BTC-26OCT24-65000-P,50.0,0.036441,51.164540,-19.107381,30.892619,19.107381,2024-10-24 17:45:02.301,671306215,2024-10-24 17:45:02.302,...,-2.083556,PUT,65000.0,2024-10-26,0.036441,-5000.0,UNKNOWN,0.032863,False,-5000.0
5,BTC-26OCT24-71000-C,40.0,-0.003578,7.247563,-1.430419,38.569581,1.430419,2024-10-24 17:45:01.938,671306214,2024-10-24 17:45:01.938,...,-0.279679,CALL,71000.0,2024-10-26,-0.003578,-5000.0,UNKNOWN,0.032863,False,-5000.0
6,BTC-28MAR25-30000-P,215.0,0.012485,3.073255,-145.876647,69.123353,145.876647,2024-10-18 15:47:31.096,671276734,2024-10-18 15:47:31.097,...,-14.682925,PUT,30000.0,2025-03-28,0.012485,-5000.0,UNKNOWN,-0.008667,False,-5000.0
7,BTC-28MAR25-200000-C,250.0,-0.021153,5.370247,-219.227460,30.772540,219.227460,2024-10-18 15:47:30.703,671276733,2024-10-18 15:47:30.703,...,-23.061732,CALL,200000.0,2025-03-28,-0.021153,-5000.0,UNKNOWN,-0.008667,False,-5000.0
8,BTC-29NOV24-52000-P,455.0,0.059127,20.977481,-335.201819,119.798181,335.201819,2024-10-18 06:00:38.554,671276193,2024-10-18 06:00:38.554,...,-24.600054,PUT,52000.0,2024-11-29,0.059127,-5000.0,UNKNOWN,0.007494,False,-5000.0
9,BTC-29NOV24-98000-C,305.0,-0.051633,21.990521,-290.742615,14.257385,290.742615,2024-10-18 06:00:38.346,671276192,2024-10-18 06:00:38.347,...,-22.102876,CALL,98000.0,2024-11-29,-0.051633,-5000.0,UNKNOWN,0.007494,False,-5000.0


In [24]:
dataframe_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   symbol            16 non-null     string        
 1   avgPrice          16 non-null     float64       
 2   delta             16 non-null     float64       
 3   theta             16 non-null     float64       
 4   positionValue     16 non-null     float64       
 5   unrealisedPnl     16 non-null     float64       
 6   markPrice         16 non-null     float64       
 7   createdTime       16 non-null     datetime64[ns]
 8   seq               16 non-null     int64         
 9   updatedTime       16 non-null     datetime64[ns]
 10  side              16 non-null     string        
 11  curRealisedPnl    16 non-null     float64       
 12  size              16 non-null     float64       
 13  positionStatus    16 non-null     string        
 14  gamma             16 non-nul

In [7]:
expiry_dataframe = dataframe_file.drop(columns=["symbol", "avgPrice","delta", "theta", "positionValue", "unrealisedPnl", "size", "markPrice",
                            "createdTime", "seq", "updatedTime","side","curRealisedPnl","positionStatus", "gamma",
                            "vega", "option_type","strike_price" ]).copy()

if round(sum(dataframe_file.total_delta), 6) == round(sum(dataframe_file.delta_hedging)/2, 6):
    print(f"Total Delta is Computed Correctly !!! , Total Options Open Positional Detla is : ",
          f"{sum(dataframe_file.total_delta)}",
          f"Total Delta Computed for Hedging : {sum(dataframe_file.delta_hedging)/2}")

else:
    print(f"ERROR !!!! Delta Computed In-Correctly",
          f"Total Delta : {sum(dataframe_file.total_delta)}",
          f"Total Delta Computed for Hedging : {sum(dataframe_file.delta_hedging)/2}")

In [25]:
Perp_Future_json = await ByBitAPI.get_PerpFutures_Position()

Parameter String is : 17298821713736TzoOeOuyIDiJN9cOu5000category=linear&baseCoin=BTC&settleCoin=USDC

Generated Signature is : f24673cba1504eb695296914f20f2c323aaafc08519a4cd8501f567aadd9d132

Perpectual Futures Position Info API Signature : f24673cba1504eb695296914f20f2c323aaafc08519a4cd8501f567aadd9d132

API Header is : {'X-BAPI-API-KEY': '6TzoOeOuyIDiJN9cOu', 'X-BAPI-SIGN': 'f24673cba1504eb695296914f20f2c323aaafc08519a4cd8501f567aadd9d132', 'X-BAPI-SIGN-TYPE': '2', 'X-BAPI-TIMESTAMP': '1729882171373', 'X-BAPI-RECV-WINDOW': '5000', 'Content-Type': 'application/json'} 

Final URL is : https://api-demo.bybit.com/v5/position/list?category=linear&baseCoin=BTC&settleCoin=USDC
Positions Info Length is : 1


In [26]:
Perp_Future_json.json()

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'nextPageCursor': 'BTCPERP%2C1729880706703%2C0',
  'category': 'linear',
  'list': [{'symbol': 'BTCPERP',
    'leverage': '',
    'autoAddMargin': 0,
    'avgPrice': '67149.6',
    'liqPrice': '',
    'riskLimitValue': '',
    'takeProfit': '',
    'positionValue': '6849.2592',
    'isReduceOnly': False,
    'tpslMode': 'Full',
    'riskId': 0,
    'trailingStop': '0',
    'unrealisedPnl': '8.21304',
    'markPrice': '67230.12',
    'adlRankIndicator': 2,
    'cumRealisedPnl': '-3.56834239',
    'positionMM': '',
    'createdTime': '1729657989639',
    'positionIdx': 0,
    'positionIM': '',
    'seq': 726586500,
    'updatedTime': '1729880706703',
    'side': 'Buy',
    'bustPrice': '',
    'positionBalance': '',
    'leverageSysUpdatedTime': '',
    'curRealisedPnl': '-3.76709256',
    'size': '0.102',
    'positionStatus': 'Normal',
    'mmrSysUpdatedTime': '',
    'stopLoss': '',
    'tradeMode': 0,
    'sessionAvgPrice': '67149.6'}]},
 '

In [30]:
PerpFutures_df = await ByBitAPI.format_perpfutures()
PerpFutures_df

,symbol,avgPrice,positionValue,unrealisedPnl,markPrice,createdTime,seq,updatedTime,side,size,positionStatus
0,BTCPERP,67149.6,6849.2592,8.21304,67230.12,2024-10-23 04:33:09.639,726586500,2024-10-25 18:25:06.703,Buy,0.102,Normal


In [31]:
float(PerpFutures_df.loc[0, "size"])

0.102

In [36]:
# PerpFutures_position = 0
if (PerpFutures_df.loc[0, "side"] == "Buy") or (PerpFutures_df.loc[0, "side"] == "BUY"):
    PerpFutures_position = float(PerpFutures_df.loc[0, "size"])
elif (PerpFutures_df.loc[0, "side"] == "Sell") or (PerpFutures_df.loc[0, "side"] == "SELL"):
    PerpFutures_position = float(PerpFutures_df.loc[0, "size"]) * (-1.0)

PerpFutures_position

0.102

In [37]:
import pandas as pd
PerFutures_position_dataframe = pd.json_normalize(data=Perp_Future_json.json()["result"]["list"])
PerFutures_position_dataframe

,symbol,leverage,autoAddMargin,avgPrice,liqPrice,riskLimitValue,takeProfit,positionValue,isReduceOnly,tpslMode,...,bustPrice,positionBalance,leverageSysUpdatedTime,curRealisedPnl,size,positionStatus,mmrSysUpdatedTime,stopLoss,tradeMode,sessionAvgPrice
0,BTCPERP,,0,67023.3,,,,67.0233,False,Full,...,,,,-0.03686282,0.001,Normal,,,0,67023.3


In [38]:
positions_labels = ["leverage", "autoAddMargin", "riskLimitValue", "takeProfit", "isReduceOnly",
                                    "tpslMode", "leverageSysUpdatedTime", "mmrSysUpdatedTime", "stopLoss", "tradeMode",
                                    "sessionAvgPrice", "trailingStop", "bustPrice", "positionBalance", "positionIdx",
                                    "positionIM", "positionMM", "adlRankIndicator", "cumRealisedPnl", "curRealisedPnl", "riskId",
                                    "liqPrice"]

In [39]:
PerFutures_position_dataframe.drop(labels=positions_labels, axis=1, inplace=True)


In [40]:
PerFutures_position_dataframe['symbol'] = PerFutures_position_dataframe['symbol'].astype(dtype="string")
PerFutures_position_dataframe['avgPrice'] = PerFutures_position_dataframe['avgPrice'].astype(dtype="float")
PerFutures_position_dataframe['positionValue'] = PerFutures_position_dataframe['positionValue'].astype(dtype="float")
PerFutures_position_dataframe['unrealisedPnl'] = PerFutures_position_dataframe['unrealisedPnl'].astype(dtype="float")
PerFutures_position_dataframe['markPrice'] = PerFutures_position_dataframe['markPrice'].astype(dtype="float")

PerFutures_position_dataframe['createdTime'] = PerFutures_position_dataframe['createdTime'].astype(dtype="float")
PerFutures_position_dataframe['createdTime'] = pd.to_datetime(PerFutures_position_dataframe['createdTime'], unit='ms')

PerFutures_position_dataframe['updatedTime'] = PerFutures_position_dataframe['updatedTime'].astype(dtype="float")
PerFutures_position_dataframe['updatedTime'] = pd.to_datetime(PerFutures_position_dataframe['updatedTime'], unit='ms')

PerFutures_position_dataframe['side'] = PerFutures_position_dataframe['side'].astype(dtype="string")
PerFutures_position_dataframe['size'] = PerFutures_position_dataframe['size'].astype(dtype="float")
PerFutures_position_dataframe['markPrice'] = PerFutures_position_dataframe['markPrice'].astype(dtype="float")

In [41]:
PerFutures_position_dataframe

,symbol,avgPrice,positionValue,unrealisedPnl,markPrice,createdTime,seq,updatedTime,side,size,positionStatus
0,BTCPERP,67023.3,67.0233,0.03763,67060.93,2024-10-23 04:33:09.639,721553256,2024-10-23 04:33:09.640,Buy,0.001,Normal


In [25]:
expiry_dataframe.drop_duplicates(subset="expiry")

,expiry,total_delta,expiry_delta,expiry_type,delta_hedging,hedging_required,PerpFutureQty
0,2024-12-27,0.097463,-5000.0,UNKNOWN,0.042414,False,-5000.0
2,2024-10-25,0.171730,-5000.0,UNKNOWN,0.093680,False,-5000.0
4,2024-10-11,0.039077,-5000.0,UNKNOWN,0.030499,False,-5000.0


In [14]:
dataframe_file

,symbol,avgPrice,delta,theta,positionValue,unrealisedPnl,markPrice,createdTime,seq,updatedTime,...,vega,option_type,strike_price,expiry,total_delta,expiry_delta,expiry_type,delta_hedging,hedging_required,PerpFutureQty
0,BTC-27DEC24-34000-P,255.0,0.033850,14.447618,-336.441063,173.558937,168.220531,2024-09-20 05:19:07.715,587548312,2024-09-20 05:19:08.362,...,-25.175638,PUT,34000.0,2024-12-27,0.067700,-5000.0,UNKNOWN,-0.042501,False,-5000.0
1,BTC-27DEC24-160000-C,155.0,-0.055100,24.596981,-506.970091,-196.970091,253.485045,2024-09-20 05:19:07.372,587548311,2024-09-20 05:19:08.122,...,-37.998693,CALL,160000.0,2024-12-27,-0.110200,-5000.0,UNKNOWN,-0.042501,False,-5000.0
2,BTC-25OCT24-48000-P,275.0,0.004633,7.609592,-11.580631,538.419369,5.790316,2024-09-20 05:19:06.361,587548308,2024-09-20 05:19:07.041,...,-1.414657,PUT,48000.0,2024-10-25,0.009265,-5000.0,UNKNOWN,-0.018541,False,-5000.0
3,BTC-25OCT24-88000-C,195.0,-0.013903,18.134528,-31.832303,358.167697,15.916152,2024-09-20 05:19:06.036,587548307,2024-09-20 05:19:06.700,...,-3.782325,CALL,88000.0,2024-10-25,-0.027806,-5000.0,UNKNOWN,-0.018541,False,-5000.0
